In [7]:
_2layer_mu0_1_multi = ['mLFR4/2layer_mu0.1_multi_Layer1_Network.txt.txt', 'mLFR4/2layer_mu0.1_multi_Layer2_Network.txt.txt']
_2layer_mu0_2_multi = ['mLFR4/2layer_mu0.2_multi_Layer1_Network.txt.txt', 'mLFR4/2layer_mu0.2_multi_Layer2_Network.txt.txt']
_2layer_mu0_3_multi = ['mLFR4/2layer_mu0.3_multi_Layer1_Network.txt.txt', 'mLFR4/2layer_mu0.3_multi_Layer2_Network.txt.txt']
_3layer_mu0_1_multi = ['mLFR4/3layer_mu0.1_multi_Layer1_Network.txt.txt', 'mLFR4/3layer_mu0.1_multi_Layer2_Network.txt.txt', 'mLFR4/3layer_mu0.1_multi_Layer3_Network.txt.txt']
_3layer_mu0_2_multi = ['mLFR4/3layer_mu0.2_multi_Layer1_Network.txt.txt', 'mLFR4/3layer_mu0.2_multi_Layer2_Network.txt.txt', 'mLFR4/3layer_mu0.2_multi_Layer3_Network.txt.txt']
_3layer_mu0_3_multi = ['mLFR4/3layer_mu0.3_multi_Layer1_Network.txt.txt', 'mLFR4/3layer_mu0.3_multi_Layer2_Network.txt.txt', 'mLFR4/3layer_mu0.3_multi_Layer3_Network.txt.txt']
_3layer_mu0_4_multi = ['mLFR4/3layer_mu0.4_multi_Layer1_Network.txt.txt', 'mLFR4/3layer_mu0.4_multi_Layer2_Network.txt.txt', 'mLFR4/3layer_mu0.4_multi_Layer3_Network.txt.txt']
_4layer_mu0_1_multi = ['mLFR4/4layer_mu0.1_multi_Layer1_Network.txt.txt', 'mLFR4/4layer_mu0.1_multi_Layer2_Network.txt.txt', 'mLFR4/4layer_mu0.1_multi_Layer3_Network.txt.txt', 'mLFR4/4layer_mu0.1_multi_Layer4_Network.txt.txt']
_4layer_mu0_2_multi = ['mLFR4/4layer_mu0.2_multi_Layer1_Network.txt.txt', 'mLFR4/4layer_mu0.2_multi_Layer2_Network.txt.txt', 'mLFR4/4layer_mu0.2_multi_Layer3_Network.txt.txt', 'mLFR4/4layer_mu0.2_multi_Layer4_Network.txt.txt']
_4layer_mu0_3_multi = ['mLFR4/4layer_mu0.3_multi_Layer1_Network.txt.txt', 'mLFR4/4layer_mu0.3_multi_Layer2_Network.txt.txt', 'mLFR4/4layer_mu0.3_multi_Layer3_Network.txt.txt', 'mLFR4/4layer_mu0.3_multi_Layer4_Network.txt.txt']
_4layer_mu0_4_multi = ['mLFR4/4layer_mu0.4_multi_Layer1_Network.txt.txt', 'mLFR4/4layer_mu0.4_multi_Layer2_Network.txt.txt', 'mLFR4/4layer_mu0.4_multi_Layer4_Network.txt.txt', 'mLFR4/4layer_mu0.4_multi_Layer4_Network.txt.txt']

testData = [_3layer_mu0_1_multi, _3layer_mu0_2_multi, _3layer_mu0_3_multi, _4layer_mu0_1_multi, _4layer_mu0_2_multi, _4layer_mu0_3_multi, _3layer_mu0_4_multi, _4layer_mu0_4_multi]
# testData = []

time: 1.15 ms


In [3]:
%load_ext autotime

import pygmo as pg
import numpy as np
import networkx as nx
import pandas as pd

import community as comm
import matplotlib.pyplot as plt

import datetime, time, os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.22 ms


In [9]:
class community_function:
  def __init__(self, fileNames, alphaSemi=None, alphaPizz=None):
    self.fileNames = fileNames
    self.G = []
    for fileName in self.fileNames:
      g = self.getNeighbourMatrixFromFile(fileName)
      self.G.append(g)
    self.dim = len(list(self.G[0].nodes))
    self.layers = len(fileNames)

    if alphaPizz is None:
      alphaPizz = 1
    self.alphaPizz = alphaPizz
    
    self.H = []
    for i in range(self.layers):
      if alphaSemi is None:
        self.alpha = np.full(self.layers, 0.5)
      else:
        self.alpha = np.full(self.layers, alphaSemi)
      H = self.mergeLayers(i, self.alpha)
      self.H.append(H)
      
  def fitness(self, x):
    d = {}
    x = np.asanyarray(x)
    for val, ind in zip(np.nditer(x), np.ndindex(x.shape[0])):
      d[ind[0]] = int(val)
      
    partitions = []
    for partitionNumber in set(d.values()):
      partition = []
      for key in d:
        if d[key] == partitionNumber:
          partition.append(key)
      partitions.append(partition)  
    
    f = 0
    for g in self.H:
      for partition in partitions:
        k_out = 0
        for node in partition:
          for i in g.neighbors(node):
            if i not in partition:
              k_out += g[node][i]['weight']

        k_in = 0
        for node in partition:
          for i in g.neighbors(node):
            if i in partition:
              k_in += g[node][i]['weight']
        k_in = k_in//2
        
        if k_in + k_out != 0:
          f += k_in/((k_in + k_out)**(self.alphaPizz))
        else:
          f += 0
    
    return [-f]

  def get_bounds(self):
    return ([1] * self.dim, [self.dim] * self.dim)
  
  def get_nix(self):
    # Number of integer dimensions (all of them)
    print(self.dim)
    return self.dim
  
  def get_name(self):
    ret = "Best partition for multiplex graph\n"
    for fileName in self.fileNames:
      ret +="File: " + fileName + "\n"
    ret += "Dimension: " + str(self.dim)
    ret += "\n\n"
    return ret
  
  def getNeighbourMatrixFromFile(self, fileName):
    A = pd.read_csv(fileName, sep=" ", header=None)
    G = nx.from_numpy_matrix(np.array(A))
    return G
  
  def drawPartition(self, graphIndex=0, partition=None, fun=nx.spring_layout, mergedDraw=False, fileNameEndsWith="_moeaGenerated.png"):
    if partition is None:
      raise ValueError("No partition passed as parameter!!")
    if type(partition) is list:
      partition = {key:val for key, val in enumerate(partition)}
    if type(partition) is not dict:
      raise ValueError("Invalid partititon as input: " + str(type(partition)))

    plt.clf()
      
    colors = self.getColors()
    if not mergedDraw:
      G = self.G[graphIndex]
    else:
      G = self.H[graphIndex]
      for i in range(self.dim):
        G.add_node(i)
    pos = fun(G)
    count = 0
    
    for com in set(partition.values()):
      toDraw = True
      list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
      labels = dict()
      for node in list_nodes:
        labels[node] = node+1
      node_colors = np.zeros((len(list_nodes), 3))
      if mergedDraw:
        try:
          node_colors[:] = colors[count]
          count += 1
        except:
          node_colors[:] = [1, 1, 1]
      else:
        if len(list_nodes) <= 3:
          node_colors[:] = [1, 1, 1]
          if fun != nx.circular_layout:
            toDraw = False
        else:
          try:
            node_colors[:] = colors[count]
            count += 1
          except:
            node_colors[:] = [1, 1, 1]
      if toDraw:
        nx.draw_networkx_nodes(G, pos, list_nodes, node_size=20, node_color=node_colors)
        nx.draw_networkx_labels(G, pos, labels, font_size=3)
    nx.draw_networkx_edges(G, pos, width=0.2)
    if mergedDraw:
      labels = nx.get_edge_attributes(G,'weight')
      nx.draw_networkx_edge_labels(G,pos,edge_labels=labels, font_size=3)
    
    if mergedDraw:
      fileNameEndsWith = "merged_view_" + fileNameEndsWith
    outfile=self.fileNames[graphIndex] + fileNameEndsWith
    plt.savefig(outfile, dpi=300)
    
  def getColors(self):
    return np.array([
            [230, 25, 75],
            [60, 180, 75],
            [255, 225, 25],
            [0, 130, 200],
            [245, 130, 48],
            [145, 30, 180],
            [70, 240, 240],
            [240, 50, 230],
            [210, 245, 60],
            [250, 190, 190],
            [0, 128, 128],
            [230, 190, 255],
            [170, 110, 40],
            [255, 250, 200],
            [128, 0, 0],
            [170, 255, 195],
            [128, 128, 0],
            [255, 215, 180],
            [0, 0, 128],
            [128, 128, 128],
            [0, 0, 0]
          ])/255
  

  def mergeLayers(self, mainLayer, alpha):
    '''
      Merge the layers into one graph but use alpha scaling for layers. The main layers scale is always 1
      
      mainLayer : integer
                index of main layer
      alpha : list
    '''
    alpha[mainLayer] = 1 #take the mainLayer's weights without scaling
    H = nx.Graph()
    for i in range(len(self.G)):
      g = self.G[i]
      for e in g.edges().data('weight'):
        (u, v, weight) = e
        if (u,v) not in H.edges():
          H.add_edge(u,v, weight=weight*alpha[i])
        else:
          H[u][v]['weight'] = H[u][v]['weight'] + weight*alpha[i]
    return H

time: 9.81 ms


In [12]:
i = 0
for data in [testData[0]]:
  now = datetime.datetime.now()
  time_now = (now.hour, now.minute, now.second)
  print(time_now)
  print("")
  outFileName = ''.join(data[0].split('_')[0:2]).split('/')[1] + '_semi_pizzuti_run'
  print(outFileName)

  problem = pg.problem(community_function(data, alphaPizz=1))
  algo = pg.algorithm(pg.sga(gen=4000))
  algo.set_verbosity(100)
  archi = pg.archipelago(n=10, algo=algo, prob=problem, pop_size=50)
  archi.evolve()
  archi.wait()
  
  for champion_x in archi.get_champions_x():
    df = pd.DataFrame()
    df[0] = list(range(0, len(champion_x)))
    df[1] = champion_x.astype(int)

    df.to_csv(outFileName + str(i) + '.txt', sep=' ', header=False, index=False)
    i += 1
  
    print(outFileName)
    print(i)
  
  now = datetime.datetime.now()
  time_now = (now.hour, now.minute, now.second)
  print(time_now)
  print("")

(21, 55, 43)

3layermu0.1_semi_pizzuti_run
128
3layermu0.1_semi_pizzuti_run
1
3layermu0.1_semi_pizzuti_run
2
3layermu0.1_semi_pizzuti_run
3
3layermu0.1_semi_pizzuti_run
4
3layermu0.1_semi_pizzuti_run
5
3layermu0.1_semi_pizzuti_run
6
3layermu0.1_semi_pizzuti_run
7
3layermu0.1_semi_pizzuti_run
8
3layermu0.1_semi_pizzuti_run
9
3layermu0.1_semi_pizzuti_run
10
(22, 14, 14)

time: 18min 31s
